In [1]:
!pip install -q cassio datasets langchain langchain-groq langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 11.1 MB/s eta 0:00:00


In [3]:
!pip install langchain-core langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


**Importing the required dependencies**

In [4]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
import cassio

In [5]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00


In [6]:
from PyPDF2 import PdfReader

## **Setup**

In [7]:
from google.colab import userdata

In [9]:
ASTRA_DB_APPLICATION_TOKEN = userdata.get('ASTRA_DB_APPLICATION_TOKEN')
ASTRA_DB_ID = userdata.get('ASTRA_DB_ID')

GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [10]:
pdf_reader = PdfReader("budget_speech.pdf")

In [11]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdf_reader.pages):
  content = page.extract_text()
  if content:
    raw_text += content

**Initialize the Connection to your database**

In [12]:
cassio.init(
    token=ASTRA_DB_APPLICATION_TOKEN,
    database_id=ASTRA_DB_ID,
)

In [13]:
llm = ChatGroq(model="gemma2-9b-it", api_key=GROQ_API_KEY)
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None
)

In [26]:
from langchain.text_splitter import CharacterTextSplitter
splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)
texts = splitter.split_text(raw_text)

In [27]:
astra_vector_store.add_texts(texts[:100])
print("Inserted %i headlines." % len(texts[:100]))
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 100 headlines.


In [28]:
first_question = True
while True:
  if first_question:
    query_text = input("\nEnter your question (or type `quit` to exit): ").strip()
  else:
    query_text = input("\nWhat's your next question (or type `quit` to exit): ").strip()

  if query_text.lower() == "quit":
    break

  if query_text == "":
    continue

  first_question = False

  print("\nQUESTION \"%s\"" % query_text)
  answer = astra_vector_index.query(query_text, llm=llm).strip()
  print("ANSWER: \"%s\"\n" % answer)

  print("FIRST DOCUMENTS BY RELEVANCE:")
  for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
    print("     [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))



Enter your question (or type `quit` to exit): How much the agriculture traget will be increased to and what the focus will be



QUESTION "How much the agriculture traget will be increased to and what the focus will be"


ANSWER: "The provided text mentions several agriculture-related goals but doesn't state a specific target increase for the agricultural sector. 

It does highlight these focuses:

* **Promoting chemical-free natural farming**, particularly around the Ganges River.
* **Increasing domestic oilseed production** to reduce reliance on imports.
* **Supporting millet production** through value addition, domestic consumption, and branding.
* **Updating agricultural university curricula** to reflect modern farming practices, including natural and zero-budget farming.


 Let me know if you have other questions about this text."

FIRST DOCUMENTS BY RELEVANCE:
     [0.7750] "comprehensive scheme to increase domestic production of oilseeds will be 
implemente ..."
     [0.7734] "Capacity Building for Infrastructure Projects  
27. With technical s upport from the ..."
     [0.7682] "of wheat and paddy from 163 lakh farmers, and ` 2.37 lakh crore direct 
payment of M ..."
     [0.7592] "of ` 44,605 c


QUESTION "quir"


ANSWER: "This document does not contain the answer to your question."

FIRST DOCUMENTS BY RELEVANCE:
     [0.6082] "sovereign Green Bonds will be issued for mobilizing resources for green 
infrastruct ..."
     [0.5778] "their activities.    
13. Hon'ble Speaker sir, Budget 2021 -22 had provided a sha rp ..."
     [0.5751] "launched for use by all central ministries for their procurements. The 
system will  ..."
     [0.5713] "to meet the needs of natural, zero -budget and organic farming, modern -day 
agricul ..."

What's your next question (or type `quit` to exit): quit
